In [1]:
import pandas as pd
import numpy as np
import os
import json

import re
from tqdm import tqdm
pd.set_option('display.max_rows', 1000+1)


In [2]:
data_dir = 'dataset/Godess_carbon/'
folder_names = os.listdir(data_dir)

In [3]:
folder = folder_names[0]
os.listdir(os.path.join(data_dir, folder))

['h_tsv_stat.txt', 'PDB.pdb', 'c_tsv_stat.txt']

In [4]:
pdb_path = os.path.join(data_dir, folder, 'PDB.pdb')

label_path_carbon = os.path.join(data_dir, folder, 'c_tsv_stat.txt')

label_path_hydrogen = os.path.join(data_dir, folder, 'h_tsv_stat.txt')

In [5]:
def experiment_process_single_pdb(f):
    temp_shift = []
    for f_l in f:
#         if 'HETATM' in f_l:
        if ('HETATM' in f_l) or ('ATOM' in f_l):
            temp_shift.append(f_l)

    new_shift = []
    for i in range(len(temp_shift)):
        temp_l = temp_shift[i].split(' ')
        temp_l_s = [i for i in temp_l if i != '']
        new_shift.append(temp_l_s[0:9] + list(temp_l_s[-2]))
            
    return new_shift

def extract_glycan_name(f):
    
    glycan_name = 'Missing glycan name'
    
    for f_l in f:
        if 'COMPND' in f_l:
            glycan_name = f_l.split('= ')[-1].split(' \n')[0]
            
    return glycan_name

def extract_process_single_label(c):
    new_g = []
    for c_l in c:
        c_temp = c_l.split('\t')

        new_l = []
        for i in c_temp:
            if '\n' in i:
                i = i.split('\n')[0]
            new_l.append(i)

        new_g.append(new_l)
        
    df = pd.DataFrame(new_g)
    
    df = df.rename(columns=df.iloc[0])
    df = df.tail(-1)
    df.index = range(len(df))
    
    return df


In [6]:
def read_connection(f):
    temp_dict = dict()
    for f_l in f:
        if 'CONECT' in f_l:
            f_l_single_space = re.sub(' +', ' ', f_l)
            f_l_list = f_l_single_space.split(' ')
            assert(f_l_list[-1] == '\n')
            key = int(f_l_list[1])
            value = f_l_list[2:-1]
            value = [int(j) for j in value]
            
            temp_dict[key] = value
    return temp_dict

In [7]:
def find_corresponding_residuals(df_pdb):
    
    corres_dict = dict()
    
    max_residual_num = np.unique(df_pdb['Residual_num'].values)
    
    for i in max_residual_num:
        value_list = df_pdb.loc[df_pdb['Residual_num'] == i]['Atom_num'].values
        corres_dict[int(i)] = list(value_list.astype(int))
    
    return corres_dict

In [8]:
def align_pdb_with_csv_by_atom_connection(df_label, df_label_hydrogen, df_pdb, dict_connect, dict_corresponding):
    dict_mapping_pdb_label = dict()

    # we assume all the root is from the first residual

    # if the first root is not the first residual, this happens for PDa cases
    # we add additional rows to the label file and make the first residual PDa to be the root
    if 'PDa' in df_pdb.loc[df_pdb['Residual_num'] == '1']['Residual_name'].values:
        df_additional_column = pd.DataFrame([[np.nan] * len(df_label.columns)], columns=df_label.columns)
        df_additional_column['Linkage'] = ''
        df_additional_column['Residue'] = 'PDa'
        
        
        df_label = pd.concat([df_additional_column, df_label])
        df_label_hydrogen = pd.concat([df_additional_column, df_label_hydrogen])
    
    
    
    dict_mapping_pdb_label[1] = df_label['Linkage'][0]
    
    # this has b
    for res_num, atom_list in sorted(dict_corresponding.items()):

        current_max_num = max(atom_list)
        current_min_num = min(atom_list)

        for atom_num in atom_list:
            current_connect_atom_list = dict_connect[atom_num]

            for connected_atom_num in current_connect_atom_list:

                if (connected_atom_num not in atom_list) and (connected_atom_num > current_max_num):

                    atom_num_name = df_pdb.loc[df_pdb['Atom_num'] == str(atom_num)]['Atom_name'].values[0]
                    connected_atom_num_name = df_pdb.loc[df_pdb['Atom_num'] == 
                                                         str(connected_atom_num)]['Atom_name'].values[0]

                    connected_atom_num_residual = df_pdb.loc[df_pdb['Atom_num'] == 
                                                             str(connected_atom_num)]['Residual_num'].values[0]

                    connected_atom_num_residual = int(connected_atom_num_residual)

                    print('atom_num:', atom_num, 'connected_atom_num:', connected_atom_num)
                    print('res_num:', res_num, 'connected_atom_num_residual:', connected_atom_num_residual)
                    print('atom_num_name:', atom_num_name, 'connected_atom_num_name:', connected_atom_num_name)
                    print('')
                    
                    assert(('O' in atom_num_name) or ('N' in atom_num_name) or ('R' in atom_num_name))
    
#                     assert('C' in connected_atom_num_name)
                    if 'O' in connected_atom_num_name:
                        continue
                    
                    if res_num == 1:
                        # if the Oxygen without a no number is the connection atom, this happens in PDa
                        if (atom_num_name == 'O') and (connected_atom_num_residual == 2):
                            new_connection_formula = df_label.loc[1, :]['Linkage']
                        else:
                            new_connection_formula = re.search('[0-9]+', atom_num_name).group()
                        dict_mapping_pdb_label[connected_atom_num_residual] = new_connection_formula
                    else:
                        new_connection_additional_component = re.search('[0-9]+', atom_num_name).group()
                        new_connection_formula = dict_mapping_pdb_label[res_num] + ',' + new_connection_additional_component
                        dict_mapping_pdb_label[connected_atom_num_residual] = new_connection_formula
        
    return dict_mapping_pdb_label, df_label, df_label_hydrogen

In [9]:
def align_pdb_with_csv_by_atom_connection_scramble_pdb(df_label, df_label_hydrogen, df_pdb, dict_connect, dict_corresponding):
    dict_mapping_pdb_label = dict()

    # we assume all the root is from the first residual

    # if the first root is not the first residual, this happens for PDa cases
    # we add additional rows to the label file and make the first residual PDa to be the root
    if 'PDa' in df_pdb.loc[df_pdb['Residual_num'] == '1']['Residual_name'].values:
        df_additional_column = pd.DataFrame([[np.nan] * len(df_label.columns)], columns=df_label.columns)
        df_additional_column['Linkage'] = ''
        df_additional_column['Residue'] = 'PDa'

        df_label = pd.concat([df_additional_column, df_label])
        df_label_hydrogen = pd.concat([df_additional_column, df_label_hydrogen])
    
    dict_mapping_pdb_label[1] = df_label['Linkage'][0]
    
    # this has b
    for res_num, atom_list in sorted(dict_corresponding.items()):

        current_max_num = max(atom_list)
        current_min_num = min(atom_list)

        for atom_num in atom_list:
            current_connect_atom_list = dict_connect[atom_num]

            for connected_atom_num in current_connect_atom_list:

#                 if (connected_atom_num not in atom_list) and (connected_atom_num > current_max_num):
                if (connected_atom_num not in atom_list):

                    atom_num_name = df_pdb.loc[df_pdb['Atom_num'] == str(atom_num)]['Atom_name'].values[0]
                    connected_atom_num_name = df_pdb.loc[df_pdb['Atom_num'] == 
                                                         str(connected_atom_num)]['Atom_name'].values[0]

                    connected_atom_num_residual = df_pdb.loc[df_pdb['Atom_num'] == 
                                                             str(connected_atom_num)]['Residual_num'].values[0]

                    connected_atom_num_residual = int(connected_atom_num_residual)

#                     print('atom_num:', atom_num, 'connected_atom_num:', connected_atom_num)
#                     print('res_num:', res_num, 'connected_atom_num_residual:', connected_atom_num_residual)
#                     print('atom_num_name:', atom_num_name, 'connected_atom_num_name:', connected_atom_num_name)
#                     print('')
                    assert( ( (('O' in atom_num_name) or ('N' in atom_num_name) or ('R' in atom_num_name)) and \
                               ('C' in connected_atom_num_name) ) or \
                              ((('O' in connected_atom_num_name) or ('N' in connected_atom_num_name) or ('R' in connected_atom_num_name)) and \
                               ('C' in atom_num_name) ) )
                    if 'O' in connected_atom_num_name:
                        continue
                    
                    if res_num == 1:
                        # if the Oxygen without a no number is the connection atom, this happens in PDa
                        if (atom_num_name == 'O') and (connected_atom_num_residual == 2):
                            new_connection_formula = df_label.loc[1, :]['Linkage']
                        else:
                            new_connection_formula = re.search('[0-9]+', atom_num_name).group()
                        dict_mapping_pdb_label[connected_atom_num_residual] = new_connection_formula
                    else:
                        new_connection_additional_component = re.search('[0-9]+', atom_num_name).group()
                        new_connection_formula = dict_mapping_pdb_label[res_num] + ',' + new_connection_additional_component
                        dict_mapping_pdb_label[connected_atom_num_residual] = new_connection_formula
        
    return dict_mapping_pdb_label, df_label, df_label_hydrogen

In [10]:
def filter_pdb(df_pdb):

    df_atom_list = df_pdb['Atom_name'].values

    ct = 0

    for atom in df_atom_list:
        if len(atom) >= 4:
            ct += 1

    fraction = ct / len(df_atom_list)

    return fraction > 0.5


In [11]:

pdb_out_dir = 'dataset/Godess_gnn/preprocess_data/Godess_carbon_unmatched_pdb/'
csv_out_carbon_dir = 'dataset/Godess_gnn/preprocess_data/Godess_carbon_unmatched_carbon_csv/'
csv_out_hydrogen_dir = 'dataset/Godess_gnn/preprocess_data/Godess_carbon_unmatched_hydrogen_csv/'


matching_out_dir = 'dataset/Godess_gnn/preprocess_data/Godess_carbon_match_dict/'
connection_out_dir = 'dataset/Godess_gnn/preprocess_data/Godess_carbon_connection_dict/'

name_out_dir = 'dataset/Godess_gnn/preprocess_data/Godess_carbon_name.csv'


i_list = []
folder_list = []
name_list = []

In [12]:
wierd_folder_list = []
# wierd_i_list = []
wierd_i_list = [61, 328, 444, 489, 554, 589, 631, 717, 739, 742, 
                958, 1045, 1128, 1251, 1314, 1380, 1388, 1395, 1532, 1535, 1677, 1813, 1817, 1830, 1883, 1923]

illegal_list = ['2132', '2135', '2077', '2062', '2092', '2052', '2122', '2125', '2124']

In [13]:

for i in tqdm(range(len(folder_names))):

# for i in tqdm(range(1980, len(folder_names))):
    
# for i in range(1):
# for i in [54]:

# for i in [163]:

# for i in [166]:

# for i in [441]:
    
# for i in [690]:

# for i in [1983]:

# for i in [690]:

# for i in wierd_i_list:

# for i in [168]:

    folder = folder_names[i]
    
    ########################################################################
    # Newly added for D2O, Glycoscience simulations, some glycan is excluded
    
    if folder in illegal_list:
        continue
    
    
    
    
    ########################################################################
#     print(i)
#     print(folder)
    pdb_path = os.path.join(data_dir, folder, 'PDB.pdb')
    
    label_path_carbon = os.path.join(data_dir, folder, 'c_tsv_stat.txt')
    
    label_path_hydrogen = os.path.join(data_dir, folder, 'h_tsv_stat.txt')
    
    # pdb shifts
    with open(pdb_path) as f:
        
        #############################################
        ####### extract pdb informations ############
        #############################################
        
        f = f.readlines()
        new_shift = experiment_process_single_pdb(f)
        g_name = extract_glycan_name(f)
        
        df_pdb = pd.DataFrame(new_shift)
        
        
        # glycan names not included, in 549
        if g_name == 'Missing glycan name':
            continue
        
        # some pdb contain redundant columns
        if df_pdb.shape[1] >= 10:
            
#             df_pdb = df_pdb.iloc[:,:-1]
            df_pdb = df_pdb.iloc[:,:10]
        
            
        df_pdb.columns = ['HETATM', 'Atom_num', 'Atom_name', 'Residual_name', 'Bound', 
                          'Residual_num', 'x', 'y', 'z', 'Atom_type']
        
        if '0' in df_pdb['Atom_type'].values:
            print('what the fuck,', i, folder)
            df_pdb_w = df_pdb
            df_pdb[['Atom_type']] = df_pdb[['Atom_type']].replace({0: 'O'})

        
        
        #############################################
        ####### extract connection info #############
        #############################################
        
        
        dict_connect = read_connection(f)
        
        
#         df_pdb_out_path = os.path.join(pdb_out_dir, str(i) + '.pdb.csv')
        
#         df_pdb.to_csv(df_pdb_out_path, index = False)
        
#         name_list.append(g_name)


        ################################################################
        ####### extract corresponding info, residual's atom ############
        ################################################################
        
        dict_corresponding = find_corresponding_residuals(df_pdb)    
        
    
    ###############################################
    ####### extract label carbon info #############
    ###############################################
    
    
    with open(label_path_carbon) as c:
        c = c.readlines()
        df_label = extract_process_single_label(c)

        # some pdb shift contain more than 12 labels, this happens mostly from the 
        if (df_label.shape[1] > 12):

            df_label = df_label.iloc[1:]
            df_label = df_label.dropna(axis=1, how='all')

        # one pdb contain only C1-C5, this happens because of the type of the monosaccharide
        elif (df_label.shape[1] < 9):
#             print(df_pdb_out_path)
            df_label['C6'] = None

#         df_label_out_path = os.path.join(csv_out_dir, str(i) + '.csv')

#         df_label.to_csv(df_label_out_path, index = False)


    
#     dict_map, df_label = align_pdb_with_csv_by_atom_connection(df_label, df_pdb, dict_connect, dict_corresponding)
    
    ###############################################
    ####### extract label hydrogen info ###########
    ###############################################
    
    with open(label_path_hydrogen) as h:
        h = h.readlines()
        
        df_label_hydrogen = extract_process_single_label(h)

        # some pdb shift contain more than 12 labels, this happens mostly from the 
        if (df_label_hydrogen.shape[1] > 12):

            df_label_hydrogen = df_label_hydrogen.iloc[1:]
            df_label_hydrogen = df_label_hydrogen.dropna(axis=1, how='all')

        # one pdb contain only C1-C5, this happens because of the type of the monosaccharide
        elif (df_label_hydrogen.shape[1] < 9):
    #             print(df_pdb_out_path)
            df_label_hydrogen['H6'] = None
    
    
    if filter_pdb(df_pdb):
        continue
        
    
    if df_pdb.loc[0, :]['Residual_num'] != '1':
        print(i, folder)
        
        dict_map, df_label, df_label_hydrogen = align_pdb_with_csv_by_atom_connection_scramble_pdb(df_label, df_label_hydrogen, df_pdb, 
                                                                                dict_connect, dict_corresponding)
    else:
        dict_map, df_label, df_label_hydrogen = align_pdb_with_csv_by_atom_connection(df_label, df_label_hydrogen, df_pdb, 
                                                                   dict_connect, dict_corresponding)
        
#         wierd_folder_list.append(folder)
#         wierd_i_list.append(i)
    ####################################################
    ##### Save pdb, label, and matching dictionary #####
    ####################################################
    
    # pdb
    df_pdb_out_path = os.path.join(pdb_out_dir, str(folder) + '.pdb.csv')
        
    df_pdb.to_csv(df_pdb_out_path, index = False)
        
    name_list.append(g_name)

    # label carbon
    df_label_out_path = os.path.join(csv_out_carbon_dir, str(folder) + '.csv')
    
    df_label.to_csv(df_label_out_path, index = False)
    
    # label hydrogen 
    
    df_label_out_path_hydrogen = os.path.join(csv_out_hydrogen_dir, str(folder) + '.csv')
    df_label_hydrogen.to_csv(df_label_out_path_hydrogen, index = False)
    
    
    i_list.append(i)
    folder_list.append(folder)

    
    # matching
    # this happens because the PDa included in the pdb file but the first row of label file 
    # is not corresponding to the second 
    if isinstance(dict_map[1], pd.core.series.Series):
        print(i, folder)
        print(dict_map[1])
        dict_map[1] = ''
    
    dict_matching_out_path = os.path.join(matching_out_dir, str(folder) + '_matching.json')
    
    with open(dict_matching_out_path, 'w') as dict_align_json:
        json.dump(dict_map, dict_align_json)
    
    # 
    dict_connection_out_path = os.path.join(connection_out_dir, str(folder) + '_connection.json')
    with open(dict_connection_out_path, 'w') as dict_connect_json:
        json.dump(dict_connect, dict_connect_json)
    
df_name = pd.DataFrame([i_list, folder_list, name_list]).T
df_name.to_csv(name_out_dir) # we need to save the index

  0%|▏                                         | 9/2416 [00:00<00:27, 86.66it/s]

atom_num: 1 connected_atom_num: 30
res_num: 1 connected_atom_num_residual: 2
atom_num_name: O4 connected_atom_num_name: C1

atom_num: 24 connected_atom_num: 46
res_num: 2 connected_atom_num_residual: 3
atom_num_name: O4 connected_atom_num_name: C1

atom_num: 27 connected_atom_num: 93
res_num: 2 connected_atom_num_residual: 5
atom_num_name: O3 connected_atom_num_name: C2

atom_num: 54 connected_atom_num: 71
res_num: 3 connected_atom_num_residual: 4
atom_num_name: O3 connected_atom_num_name: C1

atom_num: 108 connected_atom_num: 130
res_num: 5 connected_atom_num_residual: 6
atom_num_name: O8 connected_atom_num_name: C2

atom_num: 145 connected_atom_num: 167
res_num: 6 connected_atom_num_residual: 7
atom_num_name: O8 connected_atom_num_name: C2

atom_num: 2 connected_atom_num: 6
res_num: 1 connected_atom_num_residual: 2
atom_num_name: O1 connected_atom_num_name: C1

atom_num: 12 connected_atom_num: 26
res_num: 2 connected_atom_num_residual: 3
atom_num_name: O4 connected_atom_num_name: C1


  1%|▌                                        | 31/2416 [00:00<00:24, 96.50it/s]

19 2349
0    
0    
Name: Linkage, dtype: object
atom_num: 2 connected_atom_num: 6
res_num: 1 connected_atom_num_residual: 2
atom_num_name: O1 connected_atom_num_name: C1

atom_num: 13 connected_atom_num: 26
res_num: 2 connected_atom_num_residual: 3
atom_num_name: O4 connected_atom_num_name: C1

atom_num: 1 connected_atom_num: 30
res_num: 1 connected_atom_num_residual: 2
atom_num_name: O4 connected_atom_num_name: C1

atom_num: 24 connected_atom_num: 51
res_num: 2 connected_atom_num_residual: 4
atom_num_name: O4 connected_atom_num_name: C1

atom_num: 27 connected_atom_num: 44
res_num: 2 connected_atom_num_residual: 3
atom_num_name: O3 connected_atom_num_name: S1

atom_num: 59 connected_atom_num: 76
res_num: 4 connected_atom_num_residual: 5
atom_num_name: O3 connected_atom_num_name: S1

atom_num: 11 connected_atom_num: 23
res_num: 1 connected_atom_num_residual: 2
atom_num_name: O4 connected_atom_num_name: C1

atom_num: 28 connected_atom_num: 42
res_num: 2 connected_atom_num_residual: 3
a

  2%|▉                                        | 53/2416 [00:00<00:24, 95.69it/s]

...

atom_num: 1 connected_atom_num: 29
res_num: 1 connected_atom_num_residual: 2
atom_num_name: O3 connected_atom_num_name: C1

atom_num: 23 connected_atom_num: 45
res_num: 2 connected_atom_num_residual: 3
atom_num_name: O4 connected_atom_num_name: C1

atom_num: 43 connected_atom_num: 65
res_num: 3 connected_atom_num_residual: 4
atom_num_name: O2 connected_atom_num_name: C1

atom_num: 71 connected_atom_num: 90
res_num: 4 connected_atom_num_residual: 5
atom_num_name: O4 connected_atom_num_name: C1

atom_num: 95 connected_atom_num: 110
res_num: 5 connected_atom_num_residual: 6
atom_num_name: O4 connected_atom_num_name: C1

atom_num: 119 connected_atom_num: 129
res_num: 6 connected_atom_num_residual: 7
atom_num_name: O3 connected_atom_num_name: C1

atom_num: 121 connected_atom_num: 285
res_num: 6 connected_atom_num_residual: 16
atom_num_name: O2 connected_atom_num_name: C1

atom_num: 135 connected_atom_num: 156
res_num: 7 connected_atom_num_residual: 8
atom_num_name: O4 connected_atom_num_nam

 99%|██████████████████████████████████████▋| 2398/2416 [00:27<00:00, 84.16it/s]

atom_num: 1 connected_atom_num: 29
res_num: 1 connected_atom_num_residual: 2
atom_num_name: O3 connected_atom_num_name: C2

atom_num: 27 connected_atom_num: 61
res_num: 2 connected_atom_num_residual: 3
atom_num_name: O4 connected_atom_num_name: C2

atom_num: 13 connected_atom_num: 23
res_num: 1 connected_atom_num_residual: 2
atom_num_name: O4 connected_atom_num_name: C1

atom_num: 1 connected_atom_num: 45
res_num: 1 connected_atom_num_residual: 3
atom_num_name: O2 connected_atom_num_name: C1

atom_num: 13 connected_atom_num: 22
res_num: 1 connected_atom_num_residual: 2
atom_num_name: O3 connected_atom_num_name: C1

atom_num: 49 connected_atom_num: 69
res_num: 3 connected_atom_num_residual: 4
atom_num_name: O6 connected_atom_num_name: S1

atom_num: 53 connected_atom_num: 74
res_num: 3 connected_atom_num_residual: 5
atom_num_name: O3 connected_atom_num_name: C1

atom_num: 84 connected_atom_num: 95
res_num: 5 connected_atom_num_residual: 6
atom_num_name: O2 connected_atom_num_name: C1

at

100%|███████████████████████████████████████| 2416/2416 [00:27<00:00, 88.75it/s]

atom_num: 2 connected_atom_num: 6
res_num: 1 connected_atom_num_residual: 2
atom_num_name: O1 connected_atom_num_name: C1

atom_num: 16 connected_atom_num: 33
res_num: 2 connected_atom_num_residual: 3
atom_num_name: N2 connected_atom_num_name: C1

atom_num: 37 connected_atom_num: 53
res_num: 3 connected_atom_num_residual: 4
atom_num_name: O6 connected_atom_num_name: S1

atom_num: 43 connected_atom_num: 58
res_num: 3 connected_atom_num_residual: 5
atom_num_name: O2 connected_atom_num_name: C1

atom_num: 62 connected_atom_num: 78
res_num: 5 connected_atom_num_residual: 6
atom_num_name: O6 connected_atom_num_name: S1

atom_num: 68 connected_atom_num: 83
res_num: 5 connected_atom_num_residual: 7
atom_num_name: O2 connected_atom_num_name: C1

atom_num: 91 connected_atom_num: 103
res_num: 7 connected_atom_num_residual: 8
atom_num_name: O3 connected_atom_num_name: S1

atom_num: 93 connected_atom_num: 108
res_num: 7 connected_atom_num_residual: 9
atom_num_name: O2 connected_atom_num_name: C1



In [60]:
df_pdb_w

,HETATM,Atom_num,Atom_name,Residual_name,Bound,Residual_num,x,y,z,Atom_type
0,HETATM,1,O7,PDa,A,1,-2.486,3.929,0.028,O
1,HETATM,2,C7,PDa,A,1,-2.536,2.568,-0.045,C
2,HETATM,3,C6,PDa,A,1,-3.563,1.888,0.604,C
3,HETATM,4,C5,PDa,A,1,-3.614,0.500,0.515,C
4,HETATM,5,O5,PDa,A,1,-4.650,-0.178,1.089,O
5,HETATM,6,C,PDa,A,1,-2.641,-0.223,-0.205,C
6,HETATM,7,C1,PDa,A,1,-2.701,-1.611,-0.390,C
7,HETATM,8,C3,PDa,A,1,-1.798,-2.262,-1.223,C
8,HETATM,9,O3,PDa,A,1,-1.895,-3.578,-1.570,O
9,HETATM,10,C2,PDa,A,1,-0.751,-1.533,-1.825,C


In [15]:
df_pdb.shape[1]

10

In [16]:
folder

'1097'

In [17]:
df_label

,Linkage,Residue,Trust,C1,C2,C3,C4,C5,C6
0,,Me,84%,58.1,None,None,None,None,None
1,1,b-D-Fucp,64%,102.2,80.3,78.1,72.4,72.3,16.1
2,"1,2",b-D-Fucp,51%,102.1,80.3,78.1,72.4,71.7,16.1
3,"1,2,2",b-D-GlcpA,28%,101.1,72.7,76.1,77.9,76.7,175.1
4,"1,2,2,4",a-D-Fucp,73%,99.4,69.3,70.3,73.1,68.8,16.5
5,"1,2,2,3",a-D-ManpA,48%,101.7,71.1,72.4,73.3,74.5,177.0
6,"1,2,2,3,4",a-D-Glcp,54%,99.1,81.3,72.8,71.2,72.3,64.9
7,"1,2,2,3,4,6",S,,None,None,None,None,None,None
8,"1,2,2,3,4,2",b-D-GlcpN,73%,101.9,57.1,75.0,71.1,72.0,61.8
9,"1,2,2,3,4,2,2",Ac,88%,175.6,23.4,None,None,None,None


In [18]:
i

2415

In [19]:
print(wierd_i_list)

print(wierd_folder_list)

[61, 328, 444, 489, 554, 589, 631, 717, 739, 742, 958, 1045, 1128, 1251, 1314, 1380, 1388, 1395, 1532, 1535, 1677, 1813, 1817, 1830, 1883, 1923]
[]


In [20]:
# align_pdb_with_csv_by_atom_connection(df_label, df_pdb, dict_connect, dict_corresponding)[0]

In [21]:
# df_pdb.sort_values(by=['Residual_num'])

In [22]:
# dict_corresponding

In [23]:
# dict_connect

In [24]:
# df_pdb['Atom_name'].values

In [25]:
# df_label

In [26]:
# name_list

In [27]:
# i, folder

In [28]:
# pd.read_csv('dataset/Godess_carbon_gnn/preprocess_data/Godess_carbon_unmatched_csv/311.csv')

In [29]:
# with open('dataset/Godess_carbon_gnn/preprocess_data/Godess_carbon_match_dict/311_matching.json') as align_json:
#     j = json.load(align_json)
# j

In [30]:
# json.dump( dict_map, open( "file_name.json", 'w' ) )

# with open("file_name.json", 'w') as align_json:
#     json.dump(dict_map, align_json)


In [31]:
# with open('file_name.json') as align_json:
#     j = json.load(align_json)
# j

In [32]:
# df_pdb

In [33]:
# def align_pdb_with_csv_by_atom_connection(df_label, df_pdb, dict_connect, dict_corresponding):
#     dict_mapping_pdb_label = dict()

#     # we assume all the root is from 

#     dict_mapping_pdb_label[1] = df_label['Linkage'][0]
    
#     # this has b
#     for res_num, atom_list in sorted(dict_corresponding.items()):

#         current_max_num = max(atom_list)
#         current_min_num = min(atom_list)

#         for atom_num in atom_list:
#             current_connect_atom_list = dict_connect[atom_num]

#             for connected_atom_num in current_connect_atom_list:

#                 if (connected_atom_num not in atom_list) and (connected_atom_num > current_max_num):

#                     atom_num_name = df_pdb.loc[df_pdb['Atom_num'] == str(atom_num)]['Atom_name'].values[0]
#                     connected_atom_num_name = df_pdb.loc[df_pdb['Atom_num'] == 
#                                                          str(connected_atom_num)]['Atom_name'].values[0]

#                     connected_atom_num_residual = df_pdb.loc[df_pdb['Atom_num'] == 
#                                                              str(connected_atom_num)]['Residual_num'].values[0]

#                     connected_atom_num_residual = int(connected_atom_num_residual)

#                     print('atom_num:', atom_num, 'connected_atom_num:', connected_atom_num)
#                     print('res_num:', res_num, 'connected_atom_num_residual:', connected_atom_num_residual)
#                     print('atom_num_name:', atom_num_name, 'connected_atom_num_name:', connected_atom_num_name)
#                     print('')
#                     assert(('O' in atom_num_name) or ('N' in atom_num_name))
#                     if res_num == 1:
#                         new_connection_formula = re.search('[0-9]+', atom_num_name).group()
#                         dict_mapping_pdb_label[connected_atom_num_residual] = new_connection_formula
#                     else:
#                         new_connection_additional_component = re.search('[0-9]+', atom_num_name).group()
#                         new_connection_formula = dict_mapping_pdb_label[res_num] + ',' + new_connection_additional_component
#                         dict_mapping_pdb_label[connected_atom_num_residual] = new_connection_formula
        
#     return dict_mapping_pdb_label

In [34]:
# dict_map

In [35]:
# df_label

In [36]:
# df_label['Linkage'].values[0]

In [37]:
# df_pdb

In [38]:
# dict_corresponding = find_corresponding_residuals(df_pdb)    

In [39]:
# dict_connect

In [40]:
# dict_corresponding

In [41]:
# align_pdb_with_csv_by_atom_connection(df_label, df_pdb, dict_connect, dict_corresponding)

In [42]:
# df_label

In [43]:
# folder

In [44]:
df_pdb

,HETATM,Atom_num,Atom_name,Residual_name,Bound,Residual_num,x,y,z,Atom_type
0,HETATM,1,C1,MOH,A,1,2.978,-7.204,3.138,C
1,HETATM,2,O1,MOH,A,1,2.470,-7.075,1.817,O
2,HETATM,3,H1,MOH,A,1,2.716,-6.329,3.740,H
3,HETATM,4,H11,MOH,A,1,2.600,-8.114,3.614,H
4,HETATM,5,H12,MOH,A,1,4.068,-7.270,3.081,H
5,HETATM,6,C1,FCB,A,2,1.043,-6.951,1.815,C
6,HETATM,7,O5,FCB,A,2,0.472,-8.239,2.080,O
7,HETATM,8,C5,FCB,A,2,-0.954,-8.192,2.146,C
8,HETATM,9,C6,FCB,A,2,-1.440,-9.566,2.601,C
9,HETATM,10,C4,FCB,A,2,-1.494,-7.799,0.765,C


In [45]:
df_label

,Linkage,Residue,Trust,C1,C2,C3,C4,C5,C6
0,,Me,84%,58.1,None,None,None,None,None
1,1,b-D-Fucp,64%,102.2,80.3,78.1,72.4,72.3,16.1
2,"1,2",b-D-Fucp,51%,102.1,80.3,78.1,72.4,71.7,16.1
3,"1,2,2",b-D-GlcpA,28%,101.1,72.7,76.1,77.9,76.7,175.1
4,"1,2,2,4",a-D-Fucp,73%,99.4,69.3,70.3,73.1,68.8,16.5
5,"1,2,2,3",a-D-ManpA,48%,101.7,71.1,72.4,73.3,74.5,177.0
6,"1,2,2,3,4",a-D-Glcp,54%,99.1,81.3,72.8,71.2,72.3,64.9
7,"1,2,2,3,4,6",S,,None,None,None,None,None,None
8,"1,2,2,3,4,2",b-D-GlcpN,73%,101.9,57.1,75.0,71.1,72.0,61.8
9,"1,2,2,3,4,2,2",Ac,88%,175.6,23.4,None,None,None,None


In [46]:
dict_connect

{1: [2, 3, 4, 5],
 2: [1, 6],
 3: [1],
 4: [1],
 5: [1],
 6: [2, 14, 16, 7],
 7: [8, 6],
 8: [7, 9, 10, 17],
 9: [8, 18, 19, 20],
 10: [8, 11, 12, 21],
 11: [10, 22],
 12: [10, 13, 14, 23],
 13: [12, 24],
 14: [12, 15, 6, 25],
 15: [14, 26],
 16: [6],
 17: [8],
 18: [9],
 19: [9],
 20: [9],
 21: [10],
 22: [11],
 23: [12],
 24: [13],
 25: [14],
 26: [15, 34, 36, 27],
 27: [28, 26],
 28: [27, 29, 30, 37],
 29: [28, 38, 39, 40],
 30: [28, 31, 32, 41],
 31: [30, 42],
 32: [30, 33, 34, 43],
 33: [32, 44],
 34: [32, 35, 26, 45],
 35: [34, 46],
 36: [26],
 37: [28],
 38: [29],
 39: [29],
 40: [29],
 41: [30],
 42: [31],
 43: [32],
 44: [33],
 45: [34],
 46: [35, 56, 58, 47],
 47: [48, 46],
 48: [47, 49, 52, 59],
 49: [48, 50, 51],
 50: [49],
 51: [49, 60],
 52: [48, 53, 54, 61],
 53: [52, 65],
 54: [52, 55, 56, 62],
 55: [54, 86],
 56: [54, 57, 46, 63],
 57: [56, 64],
 58: [46],
 59: [48],
 60: [51],
 61: [52],
 62: [54],
 63: [56],
 64: [57],
 65: [53, 73, 75, 66],
 66: [67, 65],
 67: [66, 

In [47]:
dict_corresponding

{1: [1, 2, 3, 4, 5],
 2: [6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25],
 3: [26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45],
 4: [46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64],
 5: [65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85],
 6: [86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105],
 7: [106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125],
 8: [126, 127, 128, 129, 130],
 9: [131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  15

In [48]:
if 'PDa' in df_pdb.loc[df_pdb['Residual_num'] == '1']['Residual_name'].values:
    df_additional_column = pd.DataFrame([[np.nan] * len(df_label.columns)], columns=df_label.columns)
    df_additional_column['Linkage'] = ''
    df_additional_column['Residue'] = 'PDa'

    df_label = pd.concat([df_additional_column, df_label])
    
dict_mapping_pdb_label = dict()

# we assume all the root is from 

dict_mapping_pdb_label[1] = df_label['Linkage'][0]
# dict_mapping_pdb_label[1] = df_label['Linkage'].values[0]

for res_num, atom_list in sorted(dict_corresponding.items()):
    
    current_max_num = max(atom_list)
    current_min_num = min(atom_list)
    print(current_max_num, current_min_num)
    for atom_num in atom_list:
        current_connect_atom_list = dict_connect[atom_num]
        
        for connected_atom_num in current_connect_atom_list:
#             if (connected_atom_num not in atom_list) and (connected_atom_num > current_max_num):
            if (connected_atom_num not in atom_list):
                atom_num_name = df_pdb.loc[df_pdb['Atom_num'] == str(atom_num)]['Atom_name'].values[0]
                connected_atom_num_name = df_pdb.loc[df_pdb['Atom_num'] == 
                                                     str(connected_atom_num)]['Atom_name'].values[0]
                
                connected_atom_num_residual = df_pdb.loc[df_pdb['Atom_num'] == 
                                                         str(connected_atom_num)]['Residual_num'].values[0]
                
                connected_atom_num_residual = int(connected_atom_num_residual)
                
                print('atom_num:', atom_num, 'connected_atom_num:', connected_atom_num)
                print('res_num:', res_num, 'connected_atom_num_residual:', connected_atom_num_residual)
                print('atom_num_name:', atom_num_name, 'connected_atom_num_name:', connected_atom_num_name)
                print('')
                
                if 'O' in connected_atom_num_name:
                    continue
                
                if res_num == 1:
                    if (atom_num_name == 'O') and (connected_atom_num_residual == 2):
                        new_connection_formula = df_label.loc[1, :]['Linkage']
                    else:
                        new_connection_formula = re.search('[0-9]+', atom_num_name).group()
                    dict_mapping_pdb_label[connected_atom_num_residual] = new_connection_formula
                else:
                    new_connection_additional_component = re.search('[0-9]+', atom_num_name).group()
                    new_connection_formula = dict_mapping_pdb_label[res_num] + ',' + new_connection_additional_component
                    dict_mapping_pdb_label[connected_atom_num_residual] = new_connection_formula

5 1
atom_num: 2 connected_atom_num: 6
res_num: 1 connected_atom_num_residual: 2
atom_num_name: O1 connected_atom_num_name: C1

25 6
atom_num: 6 connected_atom_num: 2
res_num: 2 connected_atom_num_residual: 1
atom_num_name: C1 connected_atom_num_name: O1

atom_num: 15 connected_atom_num: 26
res_num: 2 connected_atom_num_residual: 3
atom_num_name: O2 connected_atom_num_name: C1

45 26
atom_num: 26 connected_atom_num: 15
res_num: 3 connected_atom_num_residual: 2
atom_num_name: C1 connected_atom_num_name: O2

atom_num: 35 connected_atom_num: 46
res_num: 3 connected_atom_num_residual: 4
atom_num_name: O2 connected_atom_num_name: C1

64 46
atom_num: 46 connected_atom_num: 35
res_num: 4 connected_atom_num_residual: 3
atom_num_name: C1 connected_atom_num_name: O2

atom_num: 53 connected_atom_num: 65
res_num: 4 connected_atom_num_residual: 5
atom_num_name: O4 connected_atom_num_name: C1

atom_num: 55 connected_atom_num: 86
res_num: 4 connected_atom_num_residual: 6
atom_num_name: O3 connected_at

In [49]:
df_label

,Linkage,Residue,Trust,C1,C2,C3,C4,C5,C6
0,,Me,84%,58.1,None,None,None,None,None
1,1,b-D-Fucp,64%,102.2,80.3,78.1,72.4,72.3,16.1
2,"1,2",b-D-Fucp,51%,102.1,80.3,78.1,72.4,71.7,16.1
3,"1,2,2",b-D-GlcpA,28%,101.1,72.7,76.1,77.9,76.7,175.1
4,"1,2,2,4",a-D-Fucp,73%,99.4,69.3,70.3,73.1,68.8,16.5
5,"1,2,2,3",a-D-ManpA,48%,101.7,71.1,72.4,73.3,74.5,177.0
6,"1,2,2,3,4",a-D-Glcp,54%,99.1,81.3,72.8,71.2,72.3,64.9
7,"1,2,2,3,4,6",S,,None,None,None,None,None,None
8,"1,2,2,3,4,2",b-D-GlcpN,73%,101.9,57.1,75.0,71.1,72.0,61.8
9,"1,2,2,3,4,2,2",Ac,88%,175.6,23.4,None,None,None,None


In [50]:
dict_mapping_pdb_label

{1: '',
 2: '1',
 3: '1,2',
 4: '1,2,2',
 5: '1,2,2,4',
 6: '1,2,2,3',
 7: '1,2,2,3,4',
 8: '1,2,2,3,4,6',
 9: '1,2,2,3,4,2'}

In [51]:
df_pdb

,HETATM,Atom_num,Atom_name,Residual_name,Bound,Residual_num,x,y,z,Atom_type
0,HETATM,1,C1,MOH,A,1,2.978,-7.204,3.138,C
1,HETATM,2,O1,MOH,A,1,2.470,-7.075,1.817,O
2,HETATM,3,H1,MOH,A,1,2.716,-6.329,3.740,H
3,HETATM,4,H11,MOH,A,1,2.600,-8.114,3.614,H
4,HETATM,5,H12,MOH,A,1,4.068,-7.270,3.081,H
5,HETATM,6,C1,FCB,A,2,1.043,-6.951,1.815,C
6,HETATM,7,O5,FCB,A,2,0.472,-8.239,2.080,O
7,HETATM,8,C5,FCB,A,2,-0.954,-8.192,2.146,C
8,HETATM,9,C6,FCB,A,2,-1.440,-9.566,2.601,C
9,HETATM,10,C4,FCB,A,2,-1.494,-7.799,0.765,C


In [52]:
dict_corresponding

{1: [1, 2, 3, 4, 5],
 2: [6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25],
 3: [26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45],
 4: [46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64],
 5: [65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85],
 6: [86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105],
 7: [106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125],
 8: [126, 127, 128, 129, 130],
 9: [131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  15

In [53]:
dict_map

{1: '',
 2: '1',
 3: '1,2',
 4: '1,2,2',
 5: '1,2,2,4',
 6: '1,2,2,3',
 7: '1,2,2,3,4',
 8: '1,2,2,3,4,6',
 9: '1,2,2,3,4,2'}

In [54]:
# atom_num_name

In [55]:
# dict_mapping_pdb_label, sorted(dict_mapping_pdb_label.items())

In [56]:
# for res_num, atom_list in dict_corresponding.items():
#     print(res_num)

In [57]:
# sorted(dict_corresponding.items())

In [58]:
# re.search('[0-9]+', 'O4').group()

In [59]:
# df_pdb.loc[df_pdb['Atom_num'] == str(1)]['Atom_name'].values[0]